**Data Preprocessing**

Steps:

1. Load subset of Kepler Data to pandas DF
2. Use lightkurve to get time series data for each item
3. Do math stuff


In [20]:
import lightkurve as lk
import matplotlib
import pandas as pd
import wotan

# KOI data csv
path_to_koi = "koi_dataset.csv"

# load to df
koi_df = pd.read_csv(path_to_koi)

row_count = len(koi_df)
# print(f"Kepler Objects of Interest Dataset Sample (5/{row_count} rows):")
# print(koi_df.head())

# debug
test_df = koi_df[1:11]

for i, row in test_df.iterrows():
    kepid = row[0]
    
    lc_search = lk.search_lightcurve(f"KIC {kepid}", mission='Kepler')

    print(lc_search)
    break

SearchResult containing 17 data products.

 #       mission      year author exptime  target_name  distance
                                     s                   arcsec 
--- ----------------- ---- ------ ------- ------------- --------
  0 Kepler Quarter 01 2009 Kepler    1800 kplr010797460      0.0
  1 Kepler Quarter 02 2009 Kepler    1800 kplr010797460      0.0
  2 Kepler Quarter 03 2009 Kepler    1800 kplr010797460      0.0
  3 Kepler Quarter 04 2010 Kepler    1800 kplr010797460      0.0
  4 Kepler Quarter 05 2010 Kepler    1800 kplr010797460      0.0
  5 Kepler Quarter 06 2010 Kepler    1800 kplr010797460      0.0
  6 Kepler Quarter 07 2010 Kepler    1800 kplr010797460      0.0
  7 Kepler Quarter 10 2011 Kepler    1800 kplr010797460      0.0
  8 Kepler Quarter 09 2011 Kepler    1800 kplr010797460      0.0
  9 Kepler Quarter 08 2011 Kepler    1800 kplr010797460      0.0
 10 Kepler Quarter 11 2012 Kepler    1800 kplr010797460      0.0
 11 Kepler Quarter 12 2012 Kepler    1800 kplr0

/var/folders/q7/5ptnw00j6v3f0kzmps66tsxw0000gn/T/ipykernel_27642/1508479901.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  kepid = row[0]
